# Measuring string similarity - what does it mean for two strings to be similar?

We are going to look at three different notions of similarity for strings of text:

 - **Length of longest common substring**
 - **Levenshtein distance** (often called "edit distance")
 - **Jaccard similarity of character shingles**

The three methods are all _character-based_ - in this notebook we won't look at anything based on words (such as similarity measures that use tf/idf). There are _many_ different ways to measure string similarity, but these three should get you started. 

You will need these things installed: `strsim`, `difflib` and `nltk`.

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)  # I want to see all my text strings, not have them annoyingly truncated


### Length of longest common substring

A conceptually simple way to measure how similar two strings are is to find the longest substring that they have in common.

For example, for the news headlines

> "With flags, song, pride, French celebrate unifying victory"

and

> "BREAKING: With flags, song, pride, French celebrate World Cup victory"

the longest common substring is

> "With flags, song, pride, French celebrate "

This kind of string matching will deal with the kind of cutting and pasting we see a lot in tweets, and in news headlines in Analytics - particularly thanks to syndication.

To find the longest common substring, we can use a `SequenceMatcher` object from the `difflib` library.

In [2]:
from difflib import SequenceMatcher

str1 = "With flags, song, pride, French celebrate unifying victory"
str2 = "BREAKING: With flags, song, pride, French celebrate World Cup victory"

# The arguments to find_longest_match tell the SequenceMatcher which sections of the strings to look in
match = SequenceMatcher(isjunk=None, a=str1, b=str2, autojunk=False).find_longest_match(0, len(str1), 0, len(str2))

print(match)

Match(a=0, b=10, size=42)


This result tells us that the longest common substring has length 42 and appears at offset 0 in the first string and offset 10 in the second string. We can use that to extract the matching bit:

In [3]:
print(str1[match.a : match.a + match.size])
print(str2[match.b : match.b + match.size])

With flags, song, pride, French celebrate 
With flags, song, pride, French celebrate 


**Note how we used `autojunk=False`**. If you don't do that, SequenceMatcher will apply some "autojunk heuristic" (which you can read about [here](https://docs.python.org/2/library/difflib.html#sequencematcher-objects)) and for strings longer than 200 characters you won't get the matches you expect.

We can wrap the above operation into some functions for reuse.

In [4]:
def longest_common_substring(s1, s2):
    match = SequenceMatcher(isjunk=None, a=s1, b=s2, autojunk=False).find_longest_match(0, len(s1), 0, len(s2))
    return s1[match.a : match.a + match.size]

# Normalise by dividing length of longest common substring by minimum of the lengths of the input strings.
# This will give us a similarity measure between 0 (no overlap) and 1 (identical).

# You could also choose to normalise by the maximum, or by the sum of the two strings.

def normalised_LCS_len(s1, s2):
    return len(longest_common_substring(s1, s2))/min(len(s1), len(s2)) if min(len(s1), len(s2)) > 0 else 0


In [5]:
print(longest_common_substring(str1, str2))
print(normalised_LCS_len(str1, str2))

With flags, song, pride, French celebrate 
0.7241379310344828


In [6]:
str1 = "Family spokesman says physicist Stephen Hawking has died at the age of 76"
str2 = "Brilliant, popular physicist Stephen Hawking has died at 76"

print(longest_common_substring(str1, str2))
normalised_LCS_len(str1, str2)

 physicist Stephen Hawking has died at 


0.6610169491525424

In [7]:
str1 = "Gatwick Airport Reopens After Drone Sightings"
str2 = "Brilliant, popular physicist Stephen Hawking has died at 76"

print(longest_common_substring(str1, str2))
normalised_LCS_len(str1, str2)

ing


0.06666666666666667

**Caveat 1:** Because this method is looking for _exact_ overlaps in the strings, it will not find cases where the strings contain parts that are "morally" the same but have changes in punctuation or minor rewordings: 

In [8]:
str1 = "Two people arrested for criminal use of drones at Gatwick airport"
str2 = "Two people arrested over 'criminal use of drones' at Gatwick Airport"

print(longest_common_substring(str1, str2))
normalised_LCS_len(str1, str2)

criminal use of drones


0.3384615384615385

**Caveat 2:** This may not behave the way you want when one of the strings is very short:

In [9]:
str1 = "Latest News"
str2 = "Latest News: Gatwick Airport in London reopens despite drone"

print(longest_common_substring(str1, str2))
normalised_LCS_len(str1, str2)

Latest News


1.0

**Caveat 3:** The basic string matching operation is case-sensitive, so you need to `casefold` your strings if you want it to be case-insensitive:

In [2]:
str1 = "British Scientist Stephen Hawking Dead At Age 76"
str2 = "British scientist Stephen Hawking dead at age 76"
str1.casefold()

'british scientist stephen hawking dead at age 76'

In [10]:
str1 = "British Scientist Stephen Hawking Dead At Age 76"
str2 = "British scientist Stephen Hawking dead at age 76"

print(longest_common_substring(str1, str2))
normalised_LCS_len(str1, str2)

print(longest_common_substring(str1.casefold(), str2.casefold()))
normalised_LCS_len(str1.casefold(), str2.casefold())


cientist Stephen Hawking 
british scientist stephen hawking dead at age 76


1.0

If you want to know about `casefold` (and why I used it instead of `.lower` or `.upper`) see here: https://stackoverflow.com/questions/45745661/python-lower-vs-casefold-in-string-matching-and-converting-to-lowercase

Here are 200 news headlines about France winning the World Cup:

In [11]:
france_df = pd.read_csv("france-football-headlines.csv")
france_df.head(15)

,text
0,How the World Cup final between France and Croatia will be won
1,How the World Cup final between France and Croatia will be won
2,Watch all the goals from the 2018 FIFA World Cup™ Final between France and Croatia | 2018 FIFA World Cup™
3,FIFA World Cup 2018 Final: 5 key moments in France’s 4-2 victory against Croatia
4,"France wins 2nd World Cup title, beats Croatia 4-2"
5,"France beats Croatia 4-2, wins 2nd World Cup title"
6,"France wins 2nd World Cup title, beats Croatia 4-2"
7,"France wins 2nd World Cup title, beats Croatia 4-2"
8,France and Croatia meet in the World Cup final
9,France overpower Croatia 4-2 to win World Cup


Let's order the headlines by how "similar" they are too one headline that we've chosen:

In [12]:
headline = "France wins 2nd World Cup title, beats Croatia 4-2"

france_df['Similarity'] = france_df['text'].apply(lambda s: normalised_LCS_len(headline.casefold(), s.casefold()))

france_df.sort_values('Similarity', ascending=False).head(15)

,text,Similarity
16,"France wins 2nd World Cup title, beats Croatia 4-2 | tbo.com",1.000000
4,"France wins 2nd World Cup title, beats Croatia 4-2",1.000000
6,"France wins 2nd World Cup title, beats Croatia 4-2",1.000000
7,"France wins 2nd World Cup title, beats Croatia 4-2",1.000000
75,"France wins 2nd World Cup title, beats Croatia 4-2 Sunday, July 15th 2018, 8:03 pm CEDT Sunday, July 15th 2018, 8:06 pm CEDT",1.000000
15,"France wins 2nd World Cup title, beats Croatia 4-2 | KFXL",1.000000
10,France Wins Second World Cup Title,0.529412
11,France wins second World Cup title,0.529412
5,"France beats Croatia 4-2, wins 2nd World Cup title",0.500000
147,BC-World Cup Advisory,0.476190


In [13]:
france_df.sort_values('Similarity').head(15)

,text,Similarity
134,The Latest: French leader to soccer team: 'Don't change',0.040000
164,Pele v Kylian Mbappe: Tale of the tape | The Northern Echo,0.040000
190,Sunday night shooting in Surrey sends man to hospital,0.040000
197,We might get Cleganebowl in 'Game of Thrones'Season 8,0.040000
184,News AskKTR Interactive Session on Twitter,0.047619
191,MCMC studies EPL direct telecast on RTM,0.051282
181,AP News in Brief at 12:04 a.m. EDT,0.058824
172,U.S. President Trump has 'low expectations'for Helsinki summit,0.060000
173,16-07-2018 - RN Breakfast - ABC Radio National (Australian Broadcasting Corporation),0.060000
174,Macron cheers from the stands — then ‘dabs’ in the changing room,0.060000


### Levenshtein distance

The _Levenshtein distance_ (also called _edit distance_) between two strings is the minimum number of edit steps required to turn one into the other, where an edit step is one of the following:

 - insert a character
 - substitute one character for another one
 - delete a character

In [14]:
from similarity.levenshtein import Levenshtein
levenshtein = Levenshtein()

str1 = "kitten"
str2 = "sitting"

print(levenshtein.distance(str1, str2))

3


You can turn "kitten" into "sitting" in three edits by:
 - substituting "s" for "k", giving "sitten"
 - substituting "i" for "e", giving "sittin"
 - insert a "g" at the end
 
There is no way to do it in fewer edits.

The `strsim` library also contains a normalised version, which divides by the length of the longest string to give a value betwen 0 and 1.

In [15]:
from similarity.normalized_levenshtein import NormalizedLevenshtein
normalized_levenshtein = NormalizedLevenshtein()

print(normalized_levenshtein.distance(str1, str2))
print(3/len("sitting"))

0.42857142857142855
0.42857142857142855


Note that this is a **distance value** not a similarity value: a value of 1 means the strings are totally different and a value of 0 means they are identical. Subtract the distance from 1 if you want a similarity value.

**Note how edit distance behaves when there are minor differences in wordings or in punctuation:**

In [16]:
str1 = "Two people arrested over 'criminal use of drones' at Gatwick Airport"
str2 = "Two people arrested for criminal use of drones at Gatwick airport"

print(levenshtein.distance(str1, str2))
print(normalized_levenshtein.distance(str1, str2))

6
0.08823529411764706


This tells us we can turn one string into the other by 6 edit steps:

 - (Delete first quote) "Two people arrested over criminal use of drones' at Gatwick Airport"
 - (Delete second quote) "Two people arrested over criminal use of drones at Gatwick Airport"
 - (Delete v character) "Two people arrested oer criminal use of drones at Gatwick Airport"
 - (Delete e character) "Two people arrested or criminal use of drones at Gatwick Airport"
 - (Insert f character) "Two people arrested for criminal use of drones at Gatwick Airport"
 - (Substitute a for A) "Two people arrested for criminal use of drones at Gatwick airport"
 
 
 

Here are 200 news headlines about drones shutting down Gatwick Airport:

In [17]:
gatwick_df = pd.read_csv("gatwick-drone-headlines.csv")
gatwick_df.head(15)

,text
0,London's Gatwick Airport resumes flights after drone chaos
1,London’s Gatwick Airport resumes flights after drone chaos
2,Gatwick Airport suspends flights after drone sightings
3,Holiday chaos as drones shut London's Gatwick Airport
4,Holiday chaos as drones shut London's Gatwick Airport
5,The Latest: Gatwick Airport in London reopens despite drone
6,Gatwick airport: How can a drone cause so much chaos?
7,UK Couple Arrested After Drone Causes Gatwick Airport Chaos And Delays
8,Two arrested over London airport drone disruption: police
9,Gatwick Airport: Why has Gatwick suspended ALL flights? When will flights resume?


Let's order the headlines by how "similar" they are too one headline that we've chosen:

In [18]:
headline = "Gatwick Airport suspends flights after drone sightings"

gatwick_df['Distance'] = gatwick_df['text'].apply(
    lambda s: normalized_levenshtein.distance(headline.casefold(), s.casefold()))

gatwick_df.sort_values('Distance').head(15)

,text,Distance
2,Gatwick Airport suspends flights after drone sightings,0.000000
37,Gatwick Airport Reopens After Drone Sightings,0.222222
65,Gatwick airport suspends flights again,0.314815
67,Gatwick airport suspends flights again,0.314815
66,Gatwick airport suspends flights again,0.314815
15,Gatwick airport reopens after drone chaos,0.351852
0,London's Gatwick Airport resumes flights after drone chaos,0.379310
1,London’s Gatwick Airport resumes flights after drone chaos,0.379310
53,Gatwick Airport Suspends Flights After Drones Reportedly Spotted Over Airfield,0.384615
96,Gatwick airport to stay closed after another drone sighting | Reuters,0.434783


In [19]:
gatwick_df.sort_values('Distance', ascending=False).head(15)

,text,Distance
153,Flughafen London-Gatwick wegen Drohnen komplett lahmgelegt,0.879310
71,Two held in Gatwick drone probe released | The Recorder,0.872727
166,"BC-EU--Britain-Gatwick A, 0375",0.870370
3,Holiday chaos as drones shut London's Gatwick Airport,0.870370
4,Holiday chaos as drones shut London's Gatwick Airport,0.870370
78,Drones shut London Gatwick airport amid busy holiday period,0.864407
141,"It was obvious even to an amoeba that sooner or later the free sale of drones to the British public would end in a disaster. I would take a bet that this attack on Gatwick airport was a trial run for a future assault, next time with a drone loaded with a f",0.863281
169,Christmas getaway drivers warned of delays | Gazette & Herald,0.852459
193,News Sniffer,0.851852
148,Drone detectors in place across UK - security minister,0.851852


**Caveat 1:** Sometimes in Analytics, a news headline will be duplicated (a content extraction issue, really). But Levenshtein distance will consider such strings as not very similar: 

In [20]:
str1 = "The world reacts to the death of physicist Stephen Hawking"
str2 = "The world reacts to the death of physicist Stephen Hawking The world reacts to the death of physicist Stephen Hawking"

print(levenshtein.distance(str1, str2))
print(normalized_levenshtein.distance(str1, str2))

59
0.5042735042735043


**Caveat 2:** When one piece of the text results from chopping up the other and reordering the pieces, Levenshtein distance will not consider them very similar.

In [21]:
str1 = "Prime Minister Theresa May emphasises 'The UK is proud to stand side by side with Israel'"
str2 = "'The UK is proud to stand side by side with Israel' emphasises Prime Minister Theresa May"

print(levenshtein.distance(str1, str2))
print(normalized_levenshtein.distance(str1, str2))

73
0.8202247191011236


### Jaccard similarity of character shingles


Given a text string, the _shingles of length $k$_ are all the substrings of length $k$:

In [22]:
from nltk import ngrams

def shingles_list(s, k):
    return list(ngrams(s, k))

def shingles_set(s, k):
    return set(ngrams(s, k))


shingles_list("hunger & thirst", 3)

[('h', 'u', 'n'),
 ('u', 'n', 'g'),
 ('n', 'g', 'e'),
 ('g', 'e', 'r'),
 ('e', 'r', ' '),
 ('r', ' ', '&'),
 (' ', '&', ' '),
 ('&', ' ', 't'),
 (' ', 't', 'h'),
 ('t', 'h', 'i'),
 ('h', 'i', 'r'),
 ('i', 'r', 's'),
 ('r', 's', 't')]

We can compare two strings `str1` and `str2` by finding the Jaccard similarity of their sets of shingles of length $k$:

 - find the shingles of `str1`
 - find the shingles of `str2`
 - divide the number of shingles with _both_ sets, by the number in _either_ set (size of intersection over size of union)

In [23]:
str1 = "hunger & thirst"
str2 = "thirst & hunger"

In [24]:
shingles1 = shingles_set(str1, 3)
shingles1

{(' ', '&', ' '),
 (' ', 't', 'h'),
 ('&', ' ', 't'),
 ('e', 'r', ' '),
 ('g', 'e', 'r'),
 ('h', 'i', 'r'),
 ('h', 'u', 'n'),
 ('i', 'r', 's'),
 ('n', 'g', 'e'),
 ('r', ' ', '&'),
 ('r', 's', 't'),
 ('t', 'h', 'i'),
 ('u', 'n', 'g')}

In [25]:
shingles2 = shingles_set(str2, 3)
shingles2

{(' ', '&', ' '),
 (' ', 'h', 'u'),
 ('&', ' ', 'h'),
 ('g', 'e', 'r'),
 ('h', 'i', 'r'),
 ('h', 'u', 'n'),
 ('i', 'r', 's'),
 ('n', 'g', 'e'),
 ('r', 's', 't'),
 ('s', 't', ' '),
 ('t', ' ', '&'),
 ('t', 'h', 'i'),
 ('u', 'n', 'g')}

In [26]:
intersection = shingles1.intersection(shingles2)
intersection

{(' ', '&', ' '),
 ('g', 'e', 'r'),
 ('h', 'i', 'r'),
 ('h', 'u', 'n'),
 ('i', 'r', 's'),
 ('n', 'g', 'e'),
 ('r', 's', 't'),
 ('t', 'h', 'i'),
 ('u', 'n', 'g')}

In [27]:
union = shingles1.union(shingles2)
union

{(' ', '&', ' '),
 (' ', 'h', 'u'),
 (' ', 't', 'h'),
 ('&', ' ', 'h'),
 ('&', ' ', 't'),
 ('e', 'r', ' '),
 ('g', 'e', 'r'),
 ('h', 'i', 'r'),
 ('h', 'u', 'n'),
 ('i', 'r', 's'),
 ('n', 'g', 'e'),
 ('r', ' ', '&'),
 ('r', 's', 't'),
 ('s', 't', ' '),
 ('t', ' ', '&'),
 ('t', 'h', 'i'),
 ('u', 'n', 'g')}

In [28]:
print(len(intersection)/len(union))
print(9/17)

0.5294117647058824
0.5294117647058824


Let's put that idea into a function:

In [29]:
def shingles_similarity(s1, s2, k):
    shingles1 = shingles_set(s1, k)
    shingles2 = shingles_set(s2, k)
    return(len(shingles1.intersection(shingles2)) / len(shingles1.union(shingles2)))


Let's revisit two of our examples from before, and see that our new similarity measure does consider them as similar:

In [30]:
str1 = "The world reacts to the death of physicist Stephen Hawking"
str2 = "The world reacts to the death of physicist Stephen Hawking The world reacts to the death of physicist Stephen Hawking"

print(shingles_similarity(str1, str2, 5))

0.9152542372881356


In [31]:
str1 = "Prime Minister Theresa May emphasises 'The UK is proud to stand side by side with Israel'"
str2 = "'The UK is proud to stand side by side with Israel' emphasises Prime Minister Theresa May"

print(shingles_similarity(str1, str2, 5))

0.8241758241758241


Here are 200 news headlines about the death of physicist Stephen Hawking:

In [32]:
stephen_hawking_df = pd.read_csv("stephen-hawking-headlines.csv")
stephen_hawking_df.head(15)

,text
0,Theoretical Physicist Stephen Hawking Dies at 76
1,Renowned Physicist Stephen Hawking dies at 76
2,Theoretical Physicist Stephen Hawking has Died at 76
3,Theoretical physicist Stephen Hawking has died at 76
4,Physicist Stephen Hawking dies at 76
5,Physicist Stephen Hawking dies at 76
6,British theoretical physicist Stephen Hawking dies at 76
7,"Brilliant, popular physicist Stephen Hawking dies at 76"
8,"Brilliant, popular physicist Stephen Hawking dies at 76"
9,"Brilliant, popular physicist Stephen Hawking dies at 76"


Let's order the headlines by how "similar" they are too one headline that we've chosen:

In [33]:
headline = "Stephen Hawking, best-known physicist of his time, has died"

stephen_hawking_df['Similarity'] = stephen_hawking_df['text'].apply(
    lambda s: shingles_similarity(headline.casefold(), s.casefold(), 5))

stephen_hawking_df.sort_values('Similarity', ascending=False).head(15)

,text,Similarity
24,"Stephen Hawking, best-known physicist of his time, has died",1.000000
28,"Stephen Hawking, best-known physicist of his time, has died",1.000000
25,"Stephen Hawking, best-known physicist of his time, has died",1.000000
26,"Stephen Hawking, best-known physicist of his time, has died",1.000000
27,"Stephen Hawking, best-known physicist of his time, has died",1.000000
47,"Stephen Hawking, best-known physicist of his time, has died - FeedPublish",0.797101
81,"Stephen Hawking, best-known physicist of his time, has died - News - McPhersonSentinel - McPherson, KS - McPherson, KS",0.591398
46,"Stephen Hawking, best-known theoretical physicist of his time, dies at 76 - Hartford Courant",0.444444
41,Renowned Physicist Stephen Hawking Has Died,0.323944
40,Renowned Physicist Stephen Hawking Has Died,0.323944


In [34]:
stephen_hawking_df.sort_values('Similarity').head(15)

,text,Similarity
199,Review: Lorde is better than ever during Oakland concert,0.0
172,Celebrate Chili's birthday with $3.13 margaritas March 13,0.0
173,Celebrate Chili's birthday with $3.13 margaritas March 13,0.0
175,Space and Earth Sciences News :: Global tributes pour in,0.0
176,The social media campaign that threw motor neurone disease into the spotlight | Epsom Guardian,0.0
167,2018-03-14: News Headlines | Positive Universe,0.0
198,Is it possible arming teachers is not a good idea?,0.0
180,Science and Faith: Beyond the Evolution and Creation Debate Online Course,0.0
181,Lok Sabha passes budget without debate; both Houses adjourn amid ruckus,0.0
182,LS passes budget without debate; both Houses adjourn amid din,0.0


**Caveat 1:** You will need to choose $k$ appropriately for your dataset; a good choice of $k$ will be affected by, among other things, the typical length of your strings. Choosing $k$ too small for your data will result in strings that you feel are very different being considered as similar. Choosing $k$ too large for your data will result in strings that you feel are similar being treated as different.


In the above, to keep the code simple, we are repeatedly computing the set of shingles of each headline. It would be more efficient to compute each one only once...

One cool thing about this similarity measure is that it can be turned into an algorithm called [Minhash](https://en.wikipedia.org/wiki/MinHash) for doing approximate deduplication in _ENORMOUS_ collections of documents.


### Final remark



We have seen three ways of measuring the similarity of text strings. There are many others, which you can read about online.

Rather than having a "one size fits all" function for string similarity that you always use, it is much better to think about what kinds of strings _YOU_ need to consider as similar for your use-case, and choosing an appropriate similarity measure based on that.